# Evn

In [1]:
import os # 파일관리
import random
import shutil
import json # json 데이터(딕셔너리) 관리
import zipfile
import math
import copy
import collections # 데이터를 모아 처리할 수 있는 유틸

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [2]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)

# One-hot

In [3]:
# one hot encoding text
text = """나는 책을 샀다
나는 책을 본다
나는 책을 팔았다
나는 책을 서점에서 샀다
나는 책을 도서관에서 본다"""

In [4]:
tokens = text.split()
tokens

['나는',
 '책을',
 '샀다',
 '나는',
 '책을',
 '본다',
 '나는',
 '책을',
 '팔았다',
 '나는',
 '책을',
 '서점에서',
 '샀다',
 '나는',
 '책을',
 '도서관에서',
 '본다']

In [5]:
words = list(dict.fromkeys(tokens))
words

['나는', '책을', '샀다', '본다', '팔았다', '서점에서', '도서관에서']

In [6]:
# 각 단어에 고유한 번호 부여한 dictionary 생성
word_to_id = {'[PAD]': 0, '[UNK]': 1}  # [PAD]: 길이 맞추는 용도, [UNK]: 알 수 없는 token
for word in words:
    word_to_id[word] = len(word_to_id)
word_to_id

{'[PAD]': 0,
 '[UNK]': 1,
 '나는': 2,
 '도서관에서': 8,
 '본다': 5,
 '샀다': 4,
 '서점에서': 7,
 '책을': 3,
 '팔았다': 6}

In [8]:
# 고유한 번호로 부터 단어를 찾을 수 있는 dictionary 생성
id_to_word = {word:_id for _id, word in word_to_id.items()}
id_to_word

{0: '[PAD]',
 1: '[UNK]',
 2: '나는',
 3: '책을',
 4: '샀다',
 5: '본다',
 6: '팔았다',
 7: '서점에서',
 8: '도서관에서'}

In [9]:
# 줄바꿈 단위로 문장 분리
sentences = text.split("\n")
sentences

['나는 책을 샀다', '나는 책을 본다', '나는 책을 팔았다', '나는 책을 서점에서 샀다', '나는 책을 도서관에서 본다']

In [10]:
# 띄어쓰기 단위로 단어 분리
tokens = []
for sentence in sentences:
    tokens.append(sentence.split())
tokens

[['나는', '책을', '샀다'],
 ['나는', '책을', '본다'],
 ['나는', '책을', '팔았다'],
 ['나는', '책을', '서점에서', '샀다'],
 ['나는', '책을', '도서관에서', '본다']]

In [11]:
# tokens을 vocabulary의 고유 번호로 변경
token_ids = []
for line_token in tokens:
    token_ids.append([word_to_id[token] for token in line_token])
token_ids

[[2, 3, 4], [2, 3, 5], [2, 3, 6], [2, 3, 7, 4], [2, 3, 8, 5]]

In [13]:
# one hot encoding
one_hot_encodings = []
for line_token in token_ids:
    print(line_token)
    one_hot_line = []  # 한 줄을 표현하는 벡터
    for id in line_token:
        ################################
        one_hot = [0] * len(word_to_id)
        one_hot[id] = 1
        one_hot_line.append(one_hot)
        ################################
    one_hot_encodings.append((one_hot_line))  # 라인을 전체 문서에 추가

[2, 3, 4]
[2, 3, 5]
[2, 3, 6]
[2, 3, 7, 4]
[2, 3, 8, 5]


In [14]:
one_hot_encodings

[[[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 1, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 1, 0],
  [0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 1, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0, 1],
  [0, 0, 0, 0, 0, 1, 0, 0, 0]]]

In [ ]:
np.argmax(np.array(one_hot_encodings[0]), axis = -1) #argmax를 통해서 one_hot을 다시 원상태로 바꿀 수 있다

In [ ]:
# tensorflow one hot
# token_ids가 앞 3개는 길이가 3 이지만 이후는 4로 tensorflow에서는 오류 발생 함
# depth는 vocabulary 크기
#############################
tf.one_hot(indices=token_ids, depth = len(word_to_id))
#############################

In [18]:
# token_ids가 앞 3개개만 one_hot으로 변경
#############################
token_ids[:3]

#############################

[[2, 3, 4], [2, 3, 5], [2, 3, 6]]

In [22]:
tf.one_hot(indices=token_ids[:3], depth =len(word_to_id))

<tf.Tensor: shape=(3, 3, 9), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.]]], dtype=float32)>

In [23]:
# 모두 길이가 4가 되도록 pad(0) 추가
pad_ids = []
for line in token_ids:
    line = line[:4]
    line += [0] * (4 - len(line))
    pad_ids.append(line)
pad_ids

[[2, 3, 4, 0], [2, 3, 5, 0], [2, 3, 6, 0], [2, 3, 7, 4], [2, 3, 8, 5]]

In [24]:
tf_one_hot_encodings = tf.one_hot(indices=pad_ids, depth=len(word_to_id))
tf_one_hot_encodings

<tf.Tensor: shape=(5, 4, 9), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.]]], dtype=float32)>

# Embedding

In [25]:
# 랜덤 매트릭스 생성
weights = np.random.randint(10, 100, size=(len(word_to_id), 4)) / 100 # 10 ~ 99까지 랜덤하게 선택하여 float형으로 변환
weights

array([[0.57, 0.93, 0.48, 0.63],
       [0.86, 0.34, 0.25, 0.59],
       [0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ],
       [0.86, 0.47, 0.44, 0.48],
       [0.77, 0.21, 0.1 , 0.85],
       [0.9 , 0.13, 0.12, 0.29],
       [0.22, 0.75, 0.85, 0.91]])

In [26]:
# 첫번째 문장 만 numpy array로 변경
one_hot_encoding_0 = np.array(one_hot_encodings[0])
one_hot_encoding_0

array([[0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [27]:
print(token_ids[0])               # [2, 3, 4]
print(weights[token_ids[0][0]])   # 2 부분
print(weights[token_ids[0][1]])   # 3 부분
print(weights[token_ids[0][2]])   # 4 부분

[2, 3, 4]
[0.33 0.36 0.4  0.53]
[0.4  0.36 0.68 0.79]
[0.9  0.83 0.57 0.6 ]


In [28]:
# one hot encoding은 matrix의 특정 row를 선택하는 것과 같은 결과
##################
np.matmul(one_hot_encoding_0, weights)
##################

array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]])

In [29]:
# tensorflow 에서도 tf.keras.layers.Embedding에서도 가능 함
# weights 초기화를 위해서 [matrix] 형태로 변환함
# embdding을 사용하면 token 번호를 바로 사용가능 함 현재 표준화된 방법
embedding = tf.keras.layers.Embedding(len(word_to_id), 4, weights=[weights])

In [32]:
token_ids[0]

[2, 3, 4]

In [30]:
embedding(np.array(token_ids[0]))

<tf.Tensor: shape=(3, 4), dtype=float32, numpy=
array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]], dtype=float32)>

In [31]:
##################
tf.gather(weights, np.array(token_ids[0]))  #임베딩 대신에 gather() 함수도 사용 가능함
##################

<tf.Tensor: shape=(3, 4), dtype=float64, numpy=
array([[0.33, 0.36, 0.4 , 0.53],
       [0.4 , 0.36, 0.68, 0.79],
       [0.9 , 0.83, 0.57, 0.6 ]])>